In [18]:
from pickle import TRUE
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
import email.utils as eutils
import time
import streamlit as st
import eppy as eppy
from eppy import modeleditor
from eppy.modeleditor import IDF
from eppy.runner.run_functions import runIDFs
import PySimpleGUI as sg
from PIL import Image, ImageTk
import os
from eppy.results import readhtml # the eppy module with functions to read the html
from eppy.results import fasthtml
import subprocess
import os
from os import system

import pprint
pp = pprint.PrettyPrinter()

In [19]:
runListPath = 'C:/Users/amitc_crl/OneDrive/Documents/GitHub/REVIVE/PhiusREVIVE/Testing/test2/runs.csv' # str(input('Select runs.csv'))

runCount = 0


#==============================================================================================================================
# 4.0 Variable Assignment
#==============================================================================================================================

runList = pd.read_csv(str(runListPath),)
runList.head()
# epwFile = runList['EPW'][runCount]


,CASE_NAME,EPW,DDY,ICFA,BEDROOMS,FLOW_COEFFICIENT [SI],EXT_WINDOW_1_U-FACTOR,EXT_WINDOW_1_SHGC,EXT_WALL_1_NAME,EXT_ROOF_1_NAME,EXT_FLOOR_1_NAME,EXT_DOOR_1_NAME,INT_FLOOR_1_NAME
0,Test 1,C:\Users\amitc_crl\OneDrive\Documents\GitHub\R...,C:\Users\amitc_crl\OneDrive\Documents\GitHub\R...,2200,5,0.05,0.15,0.35,Exterior_Wall_+1in_EPS,Exterior_Roof_R-30,Exterior_Slab_+_2in_EPS,Exterior_Door,Interior_Floor
